In [ ]:
import siibra
from packaging.version import Version
assert Version(siibra.__version__) >= Version('1.0a14')

from nilearn import plotting
import matplotlib.pyplot as plt
%matplotlib notebook

### Define two points in MNI space

We start by specifying some points in the reference space. Here, we use the MNI space. We can find such points, amongst other possibilities, by clicking them in the [siibra-explorer](https://siibra-explorer.apps.hbp.eu/). `siibra` has specific data types for points and point sets, which are aware of the reference space used, and can be warped between reference spaces on demand.

In [ ]:
# These are copy-pasted from the interactive atlas viewer:
points = siibra.PointSet(
    (
        [-25.65, -2.75, -33.75],
        [-37.35, -81.05, -6.3],
    ),
    space='mni152', sigma_mm=5)

In [ ]:
plotting.view_markers(list(map(tuple,points)), ['red', 'cyan'], marker_size=10)  

### Assign brain regions to the 3D points

We assign the points to brain regions from the Julich-Brain cytoarchitectonic atlas, using a certain location tolerance of 5mm standard deviation.

In [ ]:
julich_pmaps = siibra.get_map(
    parcellation=siibra.parcellations.get('julich'),
    space=siibra.spaces.get('mni152'),
    maptype=siibra.MapType.STATISTICAL
)
assignments = julich_pmaps.assign(points)

The result of the assignment is a pandas dataframe, where each row describes a region assigned to one of the input structures (which in this case are the provided two points, 0 and 1). Each assignment has some scores describing the relation of the point, represented as a 3D Gaussian kernel with bandwidth corresponding to the point uncertainty, and the respectivty assigned statistical map from the parcellation:

- "Value" refers to the average value of the statistcal map under the point kernel
- "Correlation" is Pearson's correlation coefficient between the statistical map and the point kernel
- "IoU" is the intersection over union between both
- "Contains" is the intersection over area of the statistical map (indicating containedness of the point)
- "Contained" is the intersection over area of the point (indicating containedness of the statistical map)


Let's look at two of the matched areas.

In [ ]:
assignments.sort_values(by='intersection over union', ascending=False, inplace=True)

a1 = assignments[assignments['input structure']==0].iloc[0]
a2 = assignments[assignments['input structure']==1].iloc[0]

for a in [a1, a2]:
    
    # fetch the probability map corresponding to the assigned regions
    statmap = julich_pmaps.fetch(region=a.region)

    # plot the map, center at the point
    view = plotting.plot_stat_map(
        statmap, 
        title=f"{a.region} ({a['correlation']:.2f})", 
        cmap='viridis', 
        cut_coords=a.centroid)

    # add the point
    view.add_markers([a.centroid])

### Find connectivity in terms of streamlines from DTI

We also investigate the connectivtiy of the first region as measured by in-vivo imaging. To do so, we select the first region in the atlas, and search for connectivity profiles.

In [ ]:
# get structural connectivity matrices for  Julich Brain
features = siibra.features.get(
    julich_pmaps.parcellation,
    siibra.features.connectivity.StreamlineCounts
)

# get one of the connectivity matrices from the first compound
sc = features[0].get_element("000")

# get the connectivity profile with the strongest 10 connections for the first matched region
profile = sc.get_profile(a1.region, max_rows=10)

profile.plot()